In [1]:
import yfinance as yf
import bs4 as bs
import pickle
import requests
import pandas as pd
import json
from dateutil import parser
from datetime import datetime
import os
import time
from pandas.io.json import json_normalize
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import wrds
from datetime import datetime as dt
import numpy as np
import glob, os
import csv
from matplotlib import pyplot as plt
pd.set_option("display.precision", 8)

import edhec_risk_kit as erk

%load_ext autoreload
%autoreload 2

In [2]:
start_download = '2006-06-01'
end_download = '2021-06-01'

In [3]:
start_date = '2009-06-30' # Define start and end dates
end_date = '2020-04-15'

### Gets currently listed SP500 companies and thier CIKs

In [4]:
# gets list of SP500 companies and their CIKs

#standard: pull from yahoo finance, yfinance
#

def save_SP500_tickers():
    resp = requests.get('https://web.archive.org/web/20200411084847/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    ciks = []
    
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        cik = row.findAll('td')[7].text[:]
        
        tickers.append(ticker)
        ciks.append(cik)
    
    #drops '\n' from certain entries in CIK list
    ciks = list(map(lambda x:x.strip(),ciks))
    
    tickers_ciks_df = pd.DataFrame({'ticker': tickers, 'cik': ciks})

        
    return tickers_ciks_df
    

### Prepping tickers list

In [5]:
tickers_ciks= save_SP500_tickers()

In [6]:
tickers = tickers_ciks['ticker'] 
tickers = [item.replace('-','.') for item in tickers]
tickers = [item.replace('.B','-A') for item in tickers]
ciks = tickers_ciks['cik']

## Remove tickers from list if change detected in change list

In [7]:
changes = pd.read_excel('sp500change.xlsx')

In [8]:
changes['Ticker'] = changes['Ticker'].astype(str)

In [9]:
changes['Ticker'] = [item.replace('-','.') for item in changes['Ticker']]
changes['Ticker'] = [item.replace('.B','-A') for item in changes['Ticker']]

In [10]:
tickers = [x for x in tickers if x not in list(changes['Ticker'])]

In [11]:
len(tickers)

232

## Returns

> ### Download Returns Data from Yfinance

In [12]:
data_yf = yf.download(tickers, start=start_download, end=end_download)

[*********************100%***********************]  232 of 232 completed

6 Failed downloads:
- ETFC: No data found, symbol may be delisted
- BLL: No data found, symbol may be delisted
- TIF: No data found, symbol may be delisted
- CTL: No data found, symbol may be delisted
- MYL: No data found, symbol may be delisted
- XLNX: No data found, symbol may be delisted


> ### Calculate Daily Returns: Training Period and Covid(testing) Period (PERCENT CHANGE)

In [13]:
#rets = (data_yf['Open'] - data_yf['Close'])/data_yf['Open']
rets_PC = data_yf['Adj Close'].pct_change()  

In [14]:
rets_PC = rets_PC.dropna(how='all',axis=0)

In [15]:
rets_PC = rets_PC.dropna(axis=1)

In [16]:
returns_PC = rets_PC[pd.to_datetime('2009-06-30', format = '%Y-%m-%d'): pd.to_datetime('2020-02-03', format = '%Y-%m-%d')]

In [17]:
returns_covid_PC = rets_PC[pd.to_datetime('2020-02-03', format = '%Y-%m-%d'):pd.to_datetime('2020-04-14', format = '%Y-%m-%d')]

> ### Calculate Daily Returns: Training Period and Covid(testing) Period (OCO)

In [18]:
rets_OCO = (data_yf['Close']-data_yf['Open'])/(data_yf['Open'])

In [19]:
rets_OCO = rets_OCO.dropna(how='all',axis=0)

In [20]:
rets_OCO = rets_OCO.dropna(axis=1)

In [21]:
returns_OCO = rets_OCO[pd.to_datetime('2009-06-30', format = '%Y-%m-%d'): pd.to_datetime('2020-02-03', format = '%Y-%m-%d')]

In [22]:
returns_covid_OCO = rets_OCO[pd.to_datetime('2020-02-03', format = '%Y-%m-%d'):pd.to_datetime('2020-04-14', format = '%Y-%m-%d')]

## Export Returns during Covid and Testing Periods

In [23]:
#Percent Change Version
returns_PC.T.to_csv('returns_PC.csv')
returns_covid_PC.T.to_csv('returns_covid_PC.csv')

In [24]:
#((Open - Close)/Open) Version
returns_OCO.T.to_csv('returns_OCO.csv')
returns_covid_OCO.T.to_csv('returns_covid_OCO.csv')

> ### Calculate Monthly Returns: Training Period and Testing Period

In [25]:
#monthly returns fr % change method
monthly_returns_PC = (returns_PC+1).groupby(pd.Grouper(freq="M")).prod() -1
monthly_returns_covid_PC = (returns_covid_PC+1).groupby(pd.Grouper(freq="M")).prod() -1

In [26]:
#monthly returns for OCO method
monthly_returns_OCO = (returns_OCO+1).groupby(pd.Grouper(freq="M")).prod() -1
monthly_returns_covid_OCO = (returns_covid_OCO+1).groupby(pd.Grouper(freq="M")).prod() -1

## T-Bills

In [27]:
t_test = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)


In [28]:
t_test = t_test.replace('.', np.nan).astype(float).ffill()/100

In [29]:
t_test['2009-06-30':'2020-02-03']

,3MO TBILL
DATE,
2009-06-30,0.0019
2009-07-01,0.0017
2009-07-02,0.0017
2009-07-03,0.0017
2009-07-06,0.0019
...,...
2020-01-28,0.0154
2020-01-29,0.0153
2020-01-30,0.0154


In [30]:
t_test = t_test/63

In [31]:
merged_test = returns_PC.merge(t_test, how = 'left', left_index=True, right_index=True)
#merged_diff = returns.merge(tbills_diff, how = 'left', left_index=True, right_index=True)

In [32]:
returns = merged_test.iloc[:,:-1].ffill().bfill()

In [33]:
t_test

,3MO TBILL
DATE,
2009-06-02,0.00002222
2009-06-03,0.00002222
2009-06-04,0.00002222
2009-06-05,0.00003016
2009-06-08,0.00003016
...,...
2020-10-12,0.00001587
2020-10-13,0.00001746
2020-10-14,0.00001905


In [34]:
t_test = pd.DataFrame(merged_test['3MO TBILL'])

In [35]:
t_test

,3MO TBILL
Date,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-06,0.00003016
2009-07-07,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [36]:
0.00024444*63

0.01539972

In [37]:
t_test = t_test.replace('.', np.nan).astype(float).ffill()/100

In [38]:
t_test['2009-06-30':'2020-02-03']

,3MO TBILL
Date,
2009-06-30,0.00000030
2009-07-01,0.00000027
2009-07-02,0.00000027
2009-07-06,0.00000030
2009-07-07,0.00000030
...,...
2020-01-28,0.00000244
2020-01-29,0.00000243
2020-01-30,0.00000244


> ### T-Bills Training

In [39]:
# read csv from file and set column name
t = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)
t.columns = ['tbill']

In [40]:
# convert from % return to decimal return
tbills = t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period
tbills = tbills/63
tbills = tbills['2009-06-30':'2020-02-03']


In [41]:
#convert from % return to decimal return
tbills_diff =  t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period, takes difference between each days rates and fills NaN values
tbills_diff=tbills_diff/63
tbills_diff = tbills_diff['2009-06-30':'2020-02-03'].diff().bfill()


In [42]:
tbills

,tbill
DATE,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-03,0.00002698
2009-07-06,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [43]:
tbills_diff

,tbill
DATE,
2009-06-30,-0.00000317
2009-07-01,-0.00000317
2009-07-02,0.00000000
2009-07-03,0.00000000
2009-07-06,0.00000317
...,...
2020-01-28,0.00000317
2020-01-29,-0.00000159
2020-01-30,0.00000159


> ### T-Bills Covid

In [44]:
# read csv from file and set column name
t = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)
t.columns = ['tbill']

In [45]:
# convert from % return to decimal return
tbills_covid = t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period
tbills_covid = tbills_covid/63
tbills_covid = tbills_covid['2020-02-03':'2020-04-14']

In [46]:
#convert from % return to decimal return
tbills_covid_diff = t.replace('.', np.nan).astype(float).ffill()/100
tbills_covid_diff = tbills_covid_diff/63
#sets date to training period, takes difference between each days rates and fills NaN values
tbills_covid_diff = tbills_covid_diff['2020-02-03':'2020-04-14'].diff().bfill()

In [47]:
tbills_covid

,tbill
DATE,
2020-02-03,0.00024444
2020-02-04,0.00024444
2020-02-05,0.00024444
2020-02-06,0.00024444
2020-02-07,0.00024286
2020-02-10,0.00024603
2020-02-11,0.00024444
2020-02-12,0.00024603
2020-02-13,0.00024762


In [48]:
tbills_covid_diff

,tbill
DATE,
2020-02-03,0.00000000
2020-02-04,0.00000000
2020-02-05,0.00000000
2020-02-06,0.00000000
2020-02-07,-0.00000159
2020-02-10,0.00000317
2020-02-11,-0.00000159
2020-02-12,0.00000159
2020-02-13,0.00000159


In [49]:
tbills_covid.shape

(52, 1)

> ### Training Period: Match Returns and T-Bills Index

In [50]:
merged = returns.merge(tbills, how = 'left', left_index=True, right_index=True)
merged_diff = returns.merge(tbills_diff, how = 'left', left_index=True, right_index=True)

In [51]:
merged

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WM,WMB,WMT,WY,XEL,XOM,XRX,YUM,ZION,tbill
Date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,-0.00587384,0.00323987,-0.01826180,-0.01631123,-0.01118105,0.00405215,-0.01761471,0.00425035,-0.02865923,-0.00200519,...,-0.01262238,-0.00191773,-0.00656283,-0.00555572,-0.00162680,-0.00949256,-0.02114764,0.00060028,-0.05555583,0.00003016
2009-07-01,0.01526340,0.00280837,0.02818520,-0.00850391,0.01272087,0.00322817,0.02913745,-0.00253957,-0.01685984,0.02008848,...,-0.00461646,-0.01473449,-0.00144542,0.01117359,0.02009766,0.00929745,0.02006171,0.05038967,-0.01816585,0.00002698
2009-07-02,-0.04364688,-0.01967384,-0.02302683,-0.00771868,-0.03558969,-0.00643609,-0.03666128,-0.03309776,-0.02947477,-0.04568711,...,-0.02247605,-0.04876458,-0.01199099,-0.04094926,-0.02502640,-0.02933647,-0.03177030,-0.02198683,-0.02555101,0.00002698
2009-07-06,-0.00862087,-0.01007018,-0.00168396,-0.00604972,-0.01736612,-0.02510100,0.00791286,0.00263319,0.00331306,0.00577817,...,-0.00255495,-0.00410132,-0.00125533,0.01626561,0.01310761,-0.00569462,-0.01874971,0.01751761,0.00632907,0.00003016
2009-07-07,-0.02864414,-0.02315848,0.02304749,-0.01434784,-0.03019145,-0.01162792,0.01794396,-0.02742877,-0.02641715,-0.02995498,...,-0.01609951,-0.05078907,0.00230500,-0.07102361,-0.02048516,-0.02261360,-0.03662434,-0.01004244,-0.00988274,0.00003016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-28,0.00716268,0.02828952,0.01816735,-0.00055798,0.02228311,0.01871498,0.00895949,0.01456707,0.02123860,0.00262144,...,-0.00115523,0.00138827,0.00638691,0.00729209,0.00448284,-0.00139021,0.04935783,0.00411931,0.01356000,0.00024444
2020-01-29,0.00068816,0.02093236,0.00841663,-0.00446729,-0.00837493,-0.00944802,-0.00546445,-0.04071803,-0.00144975,0.00871507,...,0.00346957,-0.02218107,-0.00608910,-0.00954269,0.00297529,-0.00835274,-0.00135938,0.01726759,-0.01529002,0.00024286
2020-01-30,-0.03278328,-0.00144929,-0.01090594,0.00022426,0.01444573,-0.00759511,0.04853483,0.02321094,-0.00375488,0.01086145,...,0.01061994,0.00519846,0.00595398,-0.00099654,0.01987564,0.01060682,0.00680650,-0.00028128,0.00841069,0.00024444


In [52]:
returns = merged.iloc[:,:-1].ffill().bfill()

In [53]:
tbills = pd.DataFrame(merged['tbill'])
tbills_diff = pd.DataFrame(merged_diff['tbill'])

In [54]:
returns

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WHR,WM,WMB,WMT,WY,XEL,XOM,XRX,YUM,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,-0.00587384,0.00323987,-0.01826180,-0.01631123,-0.01118105,0.00405215,-0.01761471,0.00425035,-0.02865923,-0.00200519,...,-0.02653222,-0.01262238,-0.00191773,-0.00656283,-0.00555572,-0.00162680,-0.00949256,-0.02114764,0.00060028,-0.05555583
2009-07-01,0.01526340,0.00280837,0.02818520,-0.00850391,0.01272087,0.00322817,0.02913745,-0.00253957,-0.01685984,0.02008848,...,0.05239662,-0.00461646,-0.01473449,-0.00144542,0.01117359,0.02009766,0.00929745,0.02006171,0.05038967,-0.01816585
2009-07-02,-0.04364688,-0.01967384,-0.02302683,-0.00771868,-0.03558969,-0.00643609,-0.03666128,-0.03309776,-0.02947477,-0.04568711,...,-0.02835450,-0.02247605,-0.04876458,-0.01199099,-0.04094926,-0.02502640,-0.02933647,-0.03177030,-0.02198683,-0.02555101
2009-07-06,-0.00862087,-0.01007018,-0.00168396,-0.00604972,-0.01736612,-0.02510100,0.00791286,0.00263319,0.00331306,0.00577817,...,-0.02688452,-0.00255495,-0.00410132,-0.00125533,0.01626561,0.01310761,-0.00569462,-0.01874971,0.01751761,0.00632907
2009-07-07,-0.02864414,-0.02315848,0.02304749,-0.01434784,-0.03019145,-0.01162792,0.01794396,-0.02742877,-0.02641715,-0.02995498,...,-0.01440395,-0.01609951,-0.05078907,0.00230500,-0.07102361,-0.02048516,-0.02261360,-0.03662434,-0.01004244,-0.00988274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-28,0.00716268,0.02828952,0.01816735,-0.00055798,0.02228311,0.01871498,0.00895949,0.01456707,0.02123860,0.00262144,...,0.05693845,-0.00115523,0.00138827,0.00638691,0.00729209,0.00448284,-0.00139021,0.04935783,0.00411931,0.01356000
2020-01-29,0.00068816,0.02093236,0.00841663,-0.00446729,-0.00837493,-0.00944802,-0.00546445,-0.04071803,-0.00144975,0.00871507,...,-0.01640394,0.00346957,-0.02218107,-0.00608910,-0.00954269,0.00297529,-0.00835274,-0.00135938,0.01726759,-0.01529002
2020-01-30,-0.03278328,-0.00144929,-0.01090594,0.00022426,0.01444573,-0.00759511,0.04853483,0.02321094,-0.00375488,0.01086145,...,-0.00850084,0.01061994,0.00519846,0.00595398,-0.00099654,0.01987564,0.01060682,0.00680650,-0.00028128,0.00841069


In [55]:
tbills

,tbill
Date,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-06,0.00003016
2009-07-07,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [56]:
tbills_diff

,tbill
Date,
2009-06-30,-0.00000317
2009-07-01,-0.00000317
2009-07-02,0.00000000
2009-07-06,0.00000317
2009-07-07,0.00000000
...,...
2020-01-28,0.00000317
2020-01-29,-0.00000159
2020-01-30,0.00000159


> ### Testing Period: Match Returns and T-Bills Index

In [57]:
merged_covid = returns_covid_PC.merge(tbills_covid, how = 'left', left_index=True, right_index=True)
merged_covid_diff = returns_covid_PC.merge(tbills_covid_diff, how = 'left', left_index=True, right_index=True)

In [58]:
returns_covid = merged_covid.iloc[:,:-1].ffill().bfill()

In [59]:
tbills_covid = pd.DataFrame(merged_covid['tbill']).ffill().bfill()

In [60]:
tbills_covid_diff = pd.DataFrame(merged_covid_diff['tbill']).ffill().bfill()

In [61]:
returns_covid.shape

(50, 226)

In [62]:
tbills_covid.shape

(50, 1)

In [63]:
tbills_covid_diff.shape

(50, 1)

> ### Training Period and Testing Period: Monthly T-Bills

In [64]:
tbills_monthly = (tbills+1).groupby(pd.Grouper(freq="M")).prod() -1
tbills_diff_monthly = (tbills_diff).groupby(pd.Grouper(freq="M")).prod()

tbills_covid_monthly = (tbills_covid+1).groupby(pd.Grouper(freq="M")).prod().ffill() -1
tbills_monthly_covid_diff = (tbills_covid_diff).groupby(pd.Grouper(freq="M")).mean()

## Calculate Sharpe Ratios

> ### Sharpes calcs with Excess - Direct (PC)

In [65]:
excess_returns_PC = returns_PC.sub(tbills['tbill'], axis = 0)
excess_returns_monthly_PC = monthly_returns_PC.sub(tbills_monthly['tbill'], axis = 0)

In [66]:
sharpes_PC = pd.DataFrame(excess_returns_PC.mean()/returns_PC.std())
#sharpes = pd.DataFrame(returns.mean()/returns.std())


sharpes_monthly_PC = pd.DataFrame(excess_returns_monthly_PC.mean()/monthly_returns_PC.std())
#sharpes_monthly = pd.DataFrame(monthly_returns.mean()/monthly_returns.std())

sharpes_PC = sharpes_PC*(252)**0.5
sharpes_monthly_PC = sharpes_monthly_PC*(12)**0.5

In [67]:
sharpes_PC.columns = ['sharpes']
sharpes_monthly_PC.columns = ['sharpes']

In [68]:
sharpes_PC.index.name = 'ticker'
sharpes_monthly_PC.index.name = 'ticker'

In [69]:
sharpes_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.09174995
APA,-0.08042712
OXY,0.01743228
NOV,0.05621413
XOM,0.06812820
...,...
NI,1.10630802
CMS,1.13490654
FISV,1.14623849


In [70]:
sharpes_monthly_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.08063525
DVN,-0.06774496
OXY,0.00844811
NOV,0.04341343
XOM,0.05761925
...,...
HD,1.24443377
CMS,1.26435842
NI,1.28138444


In [71]:
sharpes_PC.sort_values(by='sharpes').mean()

sharpes    0.6248492
dtype: float64

In [72]:
sharpes_monthly_PC.sort_values(by='sharpes').mean()

sharpes    0.66312193
dtype: float64

### EXPORT

In [73]:
sharpes_PC.to_csv('Export/sharpes_PC.csv')
sharpes_monthly_PC.to_csv('Export/sharpes_monthly_PC.csv')

> ### Sharpes calcs with Excess - Direct (OCO)

In [74]:
excess_returns_OCO = returns_OCO.sub(tbills['tbill'], axis = 0)
excess_returns_monthly_OCO = monthly_returns_OCO.sub(tbills_monthly['tbill'], axis = 0)

In [75]:
sharpes_OCO = pd.DataFrame(excess_returns_OCO.mean()/returns_OCO.std())
#sharpes = pd.DataFrame(returns.mean()/returns.std())


sharpes_monthly_OCO = pd.DataFrame(excess_returns_monthly_OCO.mean()/monthly_returns_OCO.std())
#sharpes_monthly = pd.DataFrame(monthly_returns.mean()/monthly_returns.std())

sharpes_OCO = sharpes_OCO*(252)**0.5
sharpes_monthly_OCO = sharpes_monthly_OCO*(12)**0.5

In [76]:
sharpes_OCO.columns = ['sharpes']
sharpes_monthly_OCO.columns = ['sharpes']

In [77]:
sharpes_OCO.index.name = 'ticker'
sharpes_monthly_OCO.index.name = 'ticker'

In [78]:
sharpes_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.84279601
BAC,-0.72981199
FCX,-0.52835816
DVN,-0.49781592
T,-0.46176320
...,...
VRSN,1.12698210
HSY,1.23984924
SHW,1.31123173


In [79]:
sharpes_monthly_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.79212502
BAC,-0.61764227
MU,-0.49637062
MRO,-0.46358456
DVN,-0.46166430
...,...
MMC,1.13392206
HSY,1.15673264
SHW,1.29382253


In [80]:
sharpes_OCO.sort_values(by='sharpes').mean()

sharpes    0.37346563
dtype: float64

In [81]:
sharpes_monthly_OCO.sort_values(by='sharpes').mean()

sharpes    0.37719524
dtype: float64

### EXPORT

In [82]:
sharpes_OCO.to_csv('Export/sharpes_OCO.csv')
sharpes_monthly_OCO.to_csv('Export/sharpes_monthly_OCO.csv')

> ### Sharpes calcs with Excess - Diff (PC)

In [83]:
excess_returns_diff_PC = returns_PC.sub(tbills_diff['tbill'], axis = 0)
excess_returns_monthly_diff_PC = monthly_returns_PC.sub(tbills_diff_monthly['tbill'], axis = 0)

In [84]:
sharpes_diff_PC = pd.DataFrame(excess_returns_diff_PC.mean()/returns_PC.std())
sharpes_monthly_diff_PC = pd.DataFrame(excess_returns_monthly_diff_PC.mean()/returns_PC.std())

sharpes_diff_PC = sharpes_diff_PC*(252)**0.5
sharpes_monthly_diff_PC = sharpes_monthly_diff_PC*(12)**0.5

In [85]:
sharpes_diff_PC.columns = ['sharpes']
sharpes_monthly_diff_PC.columns = ['sharpes']

In [86]:
sharpes_diff_PC.index.name = 'ticker'
sharpes_monthly_diff_PC.index.name = 'ticker'

In [87]:
sharpes_diff_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.03152392
APA,-0.02079243
OXY,0.10078572
NOV,0.11964176
FCX,0.13814301
...,...
NI,1.22333852
FISV,1.26122920
CMS,1.26710820


In [88]:
sharpes_monthly_diff_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.09392850
DVN,-0.05019375
OXY,0.41476761
NOV,0.47844727
MRO,0.61774843
...,...
NI,5.42916292
FISV,5.58856041
CMS,5.66943915


In [89]:
sharpes_diff_PC.sort_values(by='sharpes').mean()

sharpes    0.71938084
dtype: float64

In [90]:
sharpes_monthly_diff_PC.sort_values(by='sharpes').mean()

sharpes    3.18858851
dtype: float64

### EXPORT

In [91]:
sharpes_diff_PC.to_csv('Export/sharpes_diff_PC.csv')
sharpes_monthly_diff_PC.to_csv('Export/sharpes_monthly_diff_PC.csv')

> ### Sharpes calcs with Excess - Diff (OCO)

In [92]:
excess_returns_diff_OCO = returns_OCO.sub(tbills_diff['tbill'], axis = 0)
excess_returns_monthly_diff_OCO = monthly_returns_OCO.sub(tbills_diff_monthly['tbill'], axis = 0)

In [93]:
sharpes_diff_OCO = pd.DataFrame(excess_returns_diff_OCO.mean()/returns_OCO.std())
sharpes_monthly_diff_OCO = pd.DataFrame(excess_returns_monthly_diff_OCO.mean()/returns_OCO.std())

sharpes_diff_OCO = sharpes_diff_OCO*(252)**0.5
sharpes_monthly_diff_OCO = sharpes_monthly_diff_OCO*(12)**0.5

In [94]:
sharpes_diff_OCO.columns = ['sharpes']
sharpes_monthly_diff_OCO.columns = ['sharpes']

In [95]:
sharpes_diff_OCO.index.name = 'ticker'
sharpes_monthly_diff_OCO.index.name = 'ticker'

In [96]:
sharpes_diff_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.74986674
BAC,-0.64687686
FCX,-0.47297146
DVN,-0.42561943
MRO,-0.38360177
...,...
MMC,1.25382541
HSY,1.37910655
SHW,1.42862237


In [97]:
sharpes_monthly_diff_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-3.30387321
BAC,-2.72250945
DVN,-1.86127420
MRO,-1.74676509
FCX,-1.74344676
...,...
MMC,5.67200664
HSY,6.31312466
SHW,6.50743824


In [98]:
sharpes_diff_OCO.sort_values(by='sharpes').mean()

sharpes    0.48583356
dtype: float64

In [99]:
sharpes_monthly_diff_OCO.sort_values(by='sharpes').mean()

sharpes    2.18945196
dtype: float64

### EXPORT

In [100]:
sharpes_diff_OCO.to_csv('Export/sharpes_diff_OCO.csv')
sharpes_monthly_diff_OCO.to_csv('Export/sharpes_monthly_diff_OCO.csv')

> ### Sharpes, No Excess (PC)

In [101]:
sharpes_no_excess_PC = pd.DataFrame(returns_PC.mean()/returns_PC.std())
sharpes_monthly_no_excess_PC = pd.DataFrame(monthly_returns_PC.mean()/monthly_returns_PC.std())

sharpes_no_excess_PC = sharpes_no_excess_PC*(252)**0.5
sharpes_monthly_no_excess_PC = sharpes_monthly_no_excess_PC*(12)**0.5

In [102]:
sharpes_no_excess_PC.columns = ['sharpes']
sharpes_monthly_no_excess_PC.columns = ['sharpes']

In [103]:
sharpes_no_excess_PC.index.name = 'ticker'
sharpes_monthly_no_excess_PC.index.name = 'ticker'

In [104]:
sharpes_no_excess_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.03146992
APA,-0.02073895
OXY,0.10086047
NOV,0.11969863
FCX,0.13818399
...,...
NI,1.22344346
FISV,1.26133231
CMS,1.26722674


In [105]:
sharpes_monthly_no_excess_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.02081294
DVN,-0.01053243
OXY,0.09366183
NOV,0.10857907
MRO,0.13739681
...,...
HD,1.36109832
CMS,1.41358528
NI,1.41995429


In [106]:
sharpes_no_excess_PC.sort_values(by='sharpes').mean()

sharpes    0.7194656
dtype: float64

In [107]:
sharpes_monthly_no_excess_PC.sort_values(by='sharpes').mean()

sharpes    0.76574365
dtype: float64

### EXPORT

In [108]:
sharpes_no_excess_PC.to_csv('Export/sharpes_no_excess_PC.csv')
sharpes_monthly_no_excess_PC.to_csv('Export/sharpes_monthly_no_excess_PC.csv')

> ### Sharpes, No Excess (OCO)

In [109]:
sharpes_no_excess_OCO = pd.DataFrame(returns_OCO.mean()/returns_OCO.std())
sharpes_monthly_no_excess_OCO = pd.DataFrame(monthly_returns_OCO.mean()/monthly_returns_OCO.std())

sharpes_no_excess_OCO = sharpes_no_excess_OCO*(252)**0.5
sharpes_monthly_no_excess_OCO = sharpes_monthly_no_excess_OCO*(12)**0.5

In [110]:
sharpes_no_excess_OCO.columns = ['sharpes']
sharpes_monthly_no_excess_OCO.columns = ['sharpes']

In [111]:
sharpes_no_excess_OCO.index.name = 'ticker'
sharpes_monthly_no_excess_OCO.index.name = 'ticker'

In [112]:
sharpes_no_excess_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.74978341
BAC,-0.64680250
FCX,-0.47292179
DVN,-0.42555469
MRO,-0.38354182
...,...
MMC,1.25395210
HSY,1.37923142
SHW,1.42872764


In [113]:
sharpes_monthly_no_excess_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.70265099
BAC,-0.54278634
MU,-0.43124417
MRO,-0.39511803
DVN,-0.39270399
...,...
MMC,1.27804366
HSY,1.28506196
SHW,1.40888311


In [114]:
sharpes_no_excess_OCO.sort_values(by='sharpes').mean()

sharpes    0.48593432
dtype: float64

In [115]:
sharpes_monthly_no_excess_OCO.sort_values(by='sharpes').mean()

sharpes    0.49096692
dtype: float64

### EXPORT

In [116]:
sharpes_no_excess_OCO.to_csv('Export/sharpes_no_excess_OCO.csv')
sharpes_monthly_no_excess_OCO.to_csv('Export/sharpes_monthly_no_excess_OCO.csv')

## Export Data

> ### Returns, Covid Returns, Monthly Sharpes (using difference method, 10 YR T-Bills)

> ### Returns, Covid Returns, Daily Sharpes (using difference method, 10 YR T-Bills)

> ### Returns, Covid Returns, Monthly Sharpes (Direct method, 10 YR T-Bills)

> ### Returns, Covid Returns, Daily Sharpes (Direct method, 10 YR T-Bills)